# Ticket 5: Clasificación básica (modelo clásico)

¿Podemos distinguir entre estados de alta vs baja atención a partir de EEG? En este notebook se segmenta la señal EEG, se extraen features simples (energía en bandas delta, theta, alpha y beta) y se simulan etiquetas para clasificar cada segmento. Se entrena un modelo de Regresión Logística y se evalúa su desempeño.

In [1]:
# Importar las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import welch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Configurar Matplotlib para mostrar gráficos en línea (Jupyter/Colab)
%matplotlib inline

print('Entorno configurado y librerías importadas correctamente.')

Entorno configurado y librerías importadas correctamente.


In [2]:
# Cargar el archivo v1p.mat y asignar nombres de canales
mat_data = loadmat('v1p.mat', squeeze_me=True)
data_array = mat_data['v1p']

df = pd.DataFrame(data_array)

channel_names = ['Fz', 'Cz', 'Pz', 'C3', 'T3', 'C4', 'T4', 'Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8', 'P3', 'P4', 'T5', 'T6', 'O1', 'O2']
if df.shape[1] == len(channel_names):
    df.columns = channel_names
else:
    print('Advertencia: El número de columnas en el dataset no coincide con el número de canales esperados.')

print('Forma de la señal EEG:', df.shape)

Forma de la señal EEG: (12258, 19)


### Función para calcular la potencia en una banda de frecuencia

Utilizando la función `welch`, se estima la densidad espectral de potencia (PSD) y se integra el área bajo la curva en el rango de la banda de interés.

In [3]:
def bandpower(data, fs, band, nperseg=128):
    """Calcula la potencia de la señal en una banda de frecuencia específica.
    
    Parámetros:
      data: Señal unidimensional (array).
      fs: Frecuencia de muestreo en Hz.
      band: Tupla (f_low, f_high) que define la banda.
      nperseg: Número de muestras por segmento para welch.
    
    Retorna:
      Potencia integrada en la banda (en µV² si la señal está en µV).
    """
    f, Pxx = welch(data, fs=fs, nperseg=nperseg)
    freq_idx = (f >= band[0]) & (f <= band[1])
    return np.trapz(Pxx[freq_idx], f[freq_idx])

# Ejemplo de uso (comentado):
# power_theta = bandpower(df['Fz'].values, 128, (4, 8))
# print('Potencia en banda theta:', power_theta, 'µV²')

### Segmentación de la señal

Se segmenta la señal del canal **Fz** en ventanas de 1 segundo (128 muestras). Cada segmento representará un ejemplo para el modelo de clasificación.

In [4]:
# Parámetros de segmentación
fs = 128  # Frecuencia de muestreo
segment_length = fs  # 1 segundo = 128 muestras
num_samples = df.shape[0]
num_segments = num_samples // segment_length
print('Número de segmentos:', num_segments)

# Extraer la señal del canal Fz
signal = df['Fz'].values

# Dividir la señal en segmentos
segments = []
for i in range(num_segments):
    start = i * segment_length
    end = start + segment_length
    segments.append(signal[start:end])
segments = np.array(segments)
print('Forma de la matriz de segmentos:', segments.shape)

Número de segmentos: 95
Forma de la matriz de segmentos: (95, 128)


### Extracción de Features para cada segmento

Para cada segmento se calculará la potencia en las siguientes bandas de frecuencia:

- **Delta:** 0.5 - 4 Hz
- **Theta:** 4 - 8 Hz
- **Alpha:** 8 - 13 Hz
- **Beta:** 13 - 30 Hz

In [5]:
# Definir bandas de frecuencia
bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30)
}

# Extraer features para cada segmento
features_list = []
for seg in segments:
    features = {}
    for band_name, band_range in bands.items():
        features[band_name] = bandpower(seg, fs, band_range, nperseg=segment_length)
    features_list.append(features)

features_df = pd.DataFrame(features_list)
print('Primeros 5 registros de features extraídos:')
print(features_df.head())

C:\Users\Luna\AppData\Local\Temp\ipykernel_5652\2414210261.py:15: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(Pxx[freq_idx], f[freq_idx])


Primeros 5 registros de features extraídos:
           delta         theta        alpha         beta
0   32459.691406  16640.219482  4972.186279  6119.483767
1  245707.286133   9474.197876  3153.559708  4741.840615
2    7549.710449   3950.868713   908.005569  1339.171967
3    2155.812531   3338.061127  2545.681030  1929.724113
4   25218.478149  14299.832520  2690.533386  1543.227493


### Asignación de etiquetas simuladas

En este ejemplo se simulan etiquetas basadas en la potencia en la banda **alpha** del canal Fz. Se asume que los segmentos con una potencia en alpha superior a la mediana se corresponden con estados de **alta atención** (Control), mientras que aquellos por debajo se asocian con **baja atención** (TDAH).

In [6]:
median_alpha = features_df['alpha'].median()
print('Valor mediano de potencia en alpha:', median_alpha)

# Asignar etiquetas: 1 para alta atención (Control), 0 para baja atención (TDAH)
features_df['Label'] = (features_df['alpha'] > median_alpha).astype(int)
features_df['Label_str'] = features_df['Label'].map({1: 'Control', 0: 'TDAH'})

print('Tabla de features con etiquetas:')
print(features_df.head())

Valor mediano de potencia en alpha: 2406.608139038086
Tabla de features con etiquetas:
           delta         theta        alpha         beta  Label Label_str
0   32459.691406  16640.219482  4972.186279  6119.483767      1   Control
1  245707.286133   9474.197876  3153.559708  4741.840615      1   Control
2    7549.710449   3950.868713   908.005569  1339.171967      0      TDAH
3    2155.812531   3338.061127  2545.681030  1929.724113      1   Control
4   25218.478149  14299.832520  2690.533386  1543.227493      1   Control


### División de los datos y entrenamiento del modelo

Se separan los datos en conjuntos de entrenamiento y prueba y se entrena un modelo de Regresión Logística para predecir la etiqueta a partir de las características extraídas.

In [7]:
# Seleccionar las features y la variable objetivo
X = features_df[['delta', 'theta', 'alpha', 'beta']]
y = features_df['Label']

# Dividir en conjunto de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print('Tamaño entrenamiento:', X_train.shape, '| Tamaño prueba:', X_test.shape)

# Entrenar un modelo de Regresión Logística
model = LogisticRegression()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

print('Precisión del modelo:', accuracy_score(y_test, y_pred))
print('Matriz de confusión:')
print(confusion_matrix(y_test, y_pred))
print('Reporte de clasificación:')
print(classification_report(y_test, y_pred))

Tamaño entrenamiento: (66, 4) | Tamaño prueba: (29, 4)
Precisión del modelo: 1.0
Matriz de confusión:
[[13  0]
 [ 0 16]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00        16

    accuracy                           1.00        29
   macro avg       1.00      1.00      1.00        29
weighted avg       1.00      1.00      1.00        29



### Análisis de Coeficientes y Relevancia de Features

Se inspeccionan los coeficientes del modelo de Regresión Logística para determinar qué características tienen mayor peso en la clasificación. Por ejemplo, si el modelo asigna un coeficiente negativo a la feature `alpha`, esto sugeriría que una menor potencia en la banda alpha se asocia con baja atención (TDAH).

In [8]:
# Crear una tabla con los coeficientes del modelo
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_[0]
})

print('Coeficientes del modelo:')
print(coef_df)

Coeficientes del modelo:
  Feature  Coefficient
0   delta    -0.000227
1   theta     0.000742
2   alpha     0.082716
3    beta     0.001281


## Conclusiones e Interpretación

El modelo de Regresión Logística se entrenó y evaluó usando las características extraídas de segmentos de EEG (canal Fz). En este ejemplo simulado, se asignaron etiquetas basadas en la potencia en la banda alpha. Los resultados (precisión, matriz de confusión y reporte de clasificación) indican el desempeño del clasificador.

Además, al inspeccionar los coeficientes del modelo se puede observar qué features son más relevantes para la distinción. Por ejemplo, si el coeficiente asociado a la potencia en alpha es negativo, esto sugeriría que segmentos con menor potencia en alpha (posiblemente relacionados con TDAH o baja atención) tienen mayor probabilidad de ser clasificados en esa categoría.

**Consejo Profesional:**
La extracción de features y su uso en modelos de clasificación es un paso clave en el desarrollo de soluciones de inteligencia artificial. Aunque en este ejemplo se han simulado etiquetas, en aplicaciones reales es fundamental contar con datos etiquetados de manera confiable. Además, analiza detenidamente la importancia de cada feature para identificar posibles mejoras en la selección y transformación de datos.